In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from Causal_inference import granger_causality,add_causality_dataset, generate_synthetic
from time import time
import pmdarima as pm
from tqdm import tqdm
from statsmodels.tsa.arima_model import ARMA
from TimeSeries import TimeSeries
from copy import deepcopy

IMPORTING data

In [2]:
import os
cwd = os.getcwd()
print(cwd)

def plot(x):
    l = np.arange(x.shape[0])
    plt.plot(l, x, c='blue')
    plt.show()

/Users/ianyuen/Documents/GitHub/RA_time_series/time_series_ra/RepLearningEngine


In [3]:
dataset_names = ["FaceFour", "InlineSkate", "PickupGestureWiimoteZ", "SemgHandMovementCh2"]

to_import =  ["mixsd0.1_0.1_causaldb", "mixsd0.1_0.05_causaldb", "mixsd0.2_0.1_causaldb", "mixsd0.2_0.05_causaldb", "randomsd0.1_effectdb", "randomsd0.2_effectdb", "rwalksd0.1_effectdb", "rwalksd0.05_effectdb"]

path = cwd +"/test_files/"

In [4]:
dataset_dict = {}

for each in dataset_names:
    dataset_dict[each] = {}
    dataset_dict[each]["truemat"] = np.load(str(path+each+"_split_truemat.npy"))
    # print(dataset_dict[each]["truemat"])
    dataset_dict[each]["causaldb"] = np.load(str(path+ each+"_causaldb.npy"))
    for import_type in to_import:
        dataset_dict[each][import_type] = np.load(str(path+each+"_"+import_type+".npy"))

In [5]:
import sklearn
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

/Users/ianyuen/Documents/GitHub/RA_time_series/time_series_ra/RepLearningEngine/tigramite/independence_tests/gpdc.py:16: UserWarning: No module named 'dcor'
  warnings.warn(str(e))
/Users/ianyuen/Documents/GitHub/RA_time_series/time_series_ra/RepLearningEngine/tigramite/independence_tests/gpdc_torch.py:18: UserWarning: No module named 'dcor'
  warnings.warn(str(e))
/Users/ianyuen/Documents/GitHub/RA_time_series/time_series_ra/RepLearningEngine/tigramite/independence_tests/cmiknn.py:15: UserWarning: cannot import name 'tigramite_cython_code' from 'tigramite' (/Users/ianyuen/Documents/GitHub/RA_time_series/time_series_ra/RepLearningEngine/tigramite/__init__.py)
  warnings.warn(str(e))


In [6]:
best_gamma = 5
neighbor_param =[2, 5, 10, 100]
lag = 2
pval = 0.05

pvals = [0.01, 0.025, 0.05, 0.1]
lags = [1, 2]

In [7]:
import importlib
importlib.reload(tigramite)
import json

In [8]:
from kNN import kNN, kNN_with_pq_NCC, kNN_with_pq_SINK, MAP, avg_recall_measure
from time import time
from Causal_inference import check_with_original
import Representation

first, without Grail

In [ ]:
#package into modular funcions
#parallelize, each import_type
#each representation
#each model used
#each alpha_level
#each lag -> output

In [11]:
for each in dataset_names:
    causal = dataset_dict[each]["causaldb"]
    representation = Representation.GRAIL(kernel="SINK", d = 100, gamma = best_gamma)
    trueMat = dataset_dict[each]["truemat"]

    #parallelism here
    for import_type in to_import:
        effectdb = dataset_dict[each][import_type]
        n1 = causal.shape[0]
        n2 = effectdb.shape[0]
        TRAIN_TS, TEST_TS = representation.get_rep_train_test(effectdb, causal) #GRAIL

        for effects in ["GRAIL", "non-GRAIL"]:     #with and without GRAIL
            if effects == "GRAIL":
                TRAIN_TS, effect =  representation.get_rep_train_test(effectdb, causal)
            else:
                effect = effectdb
            rep = effects
            var_names = np.arange(len(effect))

            df = pp.DataFrame(effect.transpose(), datatime = np.arange(len(effect[0])),var_names=var_names)
            #loop through PCMI or PCMCI-plus
            parcorr = ParCorr(significance='analytic')
            pcmci = PCMCI(
                dataframe=df, 
                cond_ind_test=parcorr,
                verbosity=0)
            pcmci.verbosity = 0
            tau_max = 3     #the highest lag to go to
            #loop through PCMI or PCMCI-plus
            return_time = None
            for model_used in ["PCMCI", "PCMCI-plus"]:
                if model_used == "PCMCI":
                    t = time()
                    results = pcmci.run_pcmci(tau_max=tau_max, pc_alpha=None)
                    return_time = time() -t
                else:
                    t = time()
                    results = pcmci.run_pcmciplus(tau_min=0, tau_max=tau_max, pc_alpha=None)
                    return_time = time() -t

                dict_results={}
                dict_results["p-values"] = results['p_matrix'].round(3)
                dict_results["MCI partial correlations"] = results['val_matrix'].round(3)
                #plotting
                q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], tau_max=8, fdr_method='fdr_bh')
                dict_results["q_matrix"] = q_matrix.round(3)
                #printing significant links
                # pcmci.print_significant_links(
                #         p_matrix = results['p_matrix'], 
                #         q_matrix = q_matrix,
                #         val_matrix = results['val_matrix'],
                #         alpha_level = alpha_level)
                effect_name = each + import_type

                for alpha_level in pvals:
                    link_matrix = pcmci.return_significant_links(pq_matrix=q_matrix,
                                val_matrix=results['val_matrix'], alpha_level=alpha_level)['link_matrix']
                    dict_results["link_matrix"] = link_matrix
                    for lagged in lags:
                        compare_matrix = []
                        val_matrix = []
                        for i in range(len(link_matrix)):
                            place_holder_link = []
                            place_holder_val = []
                            for j in range(len(link_matrix[i])):
                                element1 = [link_matrix[i][j][lagged]]
                                element2 = [results['val_matrix'][i][j][lagged]]
                                place_holder_link.append(element1)
                                place_holder_val.append(element2)
                            compare_matrix.append(place_holder_link)
                            val_matrix.append(place_holder_val)
                        compare_matrix = np.array(compare_matrix)
                        val_matrix = np.array(val_matrix)
                        check_matrix = compare_matrix.reshape((trueMat.shape[0], trueMat.shape[1]))
                        check_results = check_with_original(trueMat, check_matrix) #[prec, rec, F1]
                        dict_results["precision"] = check_results[0]
                        dict_results["recall"] = check_results[1]
                        dict_results["f-score"] = check_results[2]
                        
                        tp.plot_graph(
                            val_matrix=val_matrix,
                            link_matrix=compare_matrix,
                            var_names=var_names,
                            figsize=(40, 40),
                            link_colorbar_label='cross-MCI (edges)',
                            node_colorbar_label='auto-MCI (nodes)',
                            )
                        plt.savefig(f"{cwd}/model_results/{effect_name}_P{alpha_level}_L{lagged}_{rep}_process_graph.png")
                        plt.clf()
                        plt.close()
                        tp.plot_time_series_graph(
                            figsize=(200, 200),
                            node_size=0.003,
                            val_matrix=val_matrix,
                            link_matrix=compare_matrix,
                            var_names=var_names,
                            link_colorbar_label='MCI',
                            )
                        plt.savefig(f"{cwd}/model_results/{effect_name}_P{alpha_level}_L{lagged}_{rep}_time_series_graph.png")
                        plt.clf()
                        plt.close()

                        #outputing and saving
                        with open(f'{cwd}/model_results/{effect_name}_P{alpha_level}_L{lagged}_{rep}_val_matrices_PCMCI.npy', 'wb') as f:
                            np.save(f, val_matrix)

                        with open(f'{cwd}/model_results/{effect_name}_P{alpha_level}_L{lagged}_{rep}_link_matrices.npy', 'wb') as f:
                            np.save(f, compare_matrix)

                        with open(f'{cwd}/model_results/{effect_name}_P{alpha_level}_L{lagged}_{rep}_pval_matrices.npy', 'wb') as f:
                            np.save(f, results['p_matrix'])

                        results_dict = {}
                        results_dict["data_examined"] = each
                        results_dict["effect_used"] = import_type
                        results_dict["alpha-level"] = alpha_level
                        results_dict["lag value"] = lagged
                        results_dict["representation used"] = rep
                        results_dict["precision"]= dict_results["precision"]
                        results_dict["recall"]= dict_results["recall"]
                        results_dict["f-score"]= dict_results["f-score"]
                        results_dict["return_time"] = return_time
                        with open(f'{cwd}/model_results/{effect_name}_P{alpha_level}_L{lagged}_{rep}_compared_stats.json', 'w') as f:
                            json.dump(results_dict, f)


/Users/ianyuen/Documents/GitHub/RA_time_series/time_series_ra/RepLearningEngine/tigramite/plotting.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  ((i % max_lag) - posarray.min(axis=0)[0])


In [12]:
print(dict_results["f-score"])
print(lagged)
print(str(return_time))
# print("true: ")
# print(trueMat.shape)
# print(trueMat.shape[0])
# print("result: ")
# print(len(compare_matrix[0]))
# print(len(compare_matrix))
# compare_matrix = compare_matrix.reshape((len(compare_matrix), len(compare_matrix[0])))
# print(compare_matrix.shape)

0.6966292134831461
1
9476.000649929047


In [9]:

for each in dataset_names:
    causal = dataset_dict[each]["causaldb"]
    representation = Representation.GRAIL(kernel="SINK", d = 100, gamma = best_gamma)
    trueMat = dataset_dict[each]["truemat"]

    for import_type in to_import:
        effectdb = dataset_dict[each][import_type]
        n1 = causal.shape[0]
        n2 = effectdb.shape[0]
        TRAIN_TS, TEST_TS = representation.get_rep_train_test(effectdb, causal) #GRAIL

        counter = 0
        for effect in [effectdb, TRAIN_TS]:     #with and without GRAIL
            var_names = np.arange(len(effect))

            df = pp.DataFrame(effect.transpose(), datatime = np.arange(len(effect[0])),var_names=var_names)
            t = time()
            parcorr = ParCorr(significance='analytic')
            pcmci = PCMCI(
                dataframe=df, 
                cond_ind_test=parcorr,
                verbosity=0)
            pcmci.verbosity = 0
            results = pcmci.run_pcmci(tau_max=8, pc_alpha=None)
            return_time = time() -t

            dict_results={}
            dict_results["p-values"] = results['p_matrix'].round(3)
            dict_results["MCI partial correlations"] = results['val_matrix'].round(3)
            #plotting
            q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], tau_max=8, fdr_method='fdr_bh')
            dict_results["q_matrix"] = q_matrix.round(3)
            #printing significant links
            # pcmci.print_significant_links(
            #         p_matrix = results['p_matrix'], 
            #         q_matrix = q_matrix,
            #         val_matrix = results['val_matrix'],
            #         alpha_level = alpha_level)
            effect_name = each + import_type

            if counter == 0:
                rep = "G"
            else:
                rep = "NG"
            for alpha_level in pvals:
                link_matrix = pcmci.return_significant_links(pq_matrix=q_matrix,
                            val_matrix=results['val_matrix'], alpha_level=alpha_level)['link_matrix']
                dict_results["link_matrix"] = link_matrix
                for lagged in lags:
                    compare_matrix = []
                    val_matrix = []
                    for i in range(len(link_matrix)):
                        place_holder_link = []
                        place_holder_val = []
                        for j in range(len(link_matrix[i])):
                            element1 = [link_matrix[i][j][lagged]]
                            element2 = [results['val_matrix'][i][j][lagged]]
                            place_holder_link.append(element1)
                            place_holder_val.append(element2)
                        compare_matrix.append(place_holder_link)
                        val_matrix.append(place_holder_val)
                    compare_matrix = np.array(compare_matrix)
                    val_matrix = np.array(val_matrix)
                    check_results = check_with_original(trueMat, compare_matrix) #[prec, rec, F1]
                    # dict_results["precision"] = check_results[0]
                    # dict_results["recall"] = check_results[1]
                    # dict_results["f-score"] = check_results[2]
                    
                    tp.plot_graph(
                        val_matrix=val_matrix,
                        link_matrix=compare_matrix,
                        var_names=var_names,
                        figsize=(40, 40),
                        link_colorbar_label='cross-MCI (edges)',
                        node_colorbar_label='auto-MCI (nodes)',
                        )
                    plt.savefig(f"{effect_name}_P{alpha_level}_L{lagged}_process_graph.png")
                    plt.clf()
                    plt.close()
                    tp.plot_time_series_graph(
                        figsize=(200, 200),
                        node_size=0.003,
                        val_matrix=val_matrix,
                        link_matrix=compare_matrix,
                        var_names=var_names,
                        link_colorbar_label='MCI',
                        )
                    plt.savefig(f"{effect_name}_P{alpha_level}_L{lagged}_{rep}_time_series_graph.png")
                    plt.clf()
                    plt.close()

                    #outputing and saving
                    with open(f'{effect_name}_P{alpha_level}_L{lagged}_{rep}_val_matrices_PCMCI.npy', 'wb') as f:
                        np.save(f, val_matrix)

                    with open(f'{effect_name}_P{alpha_level}_L{lagged}_{rep}_link_matrices.npy', 'wb') as f:
                        np.save(f, compare_matrix)

                    with open(f'{effect_name}_P{alpha_level}_L{lagged}_{rep}_pval_matrices.npy', 'wb') as f:
                        np.save(f, results['p_matrix'])

                    with open(f'{effect_name}_P{alpha_level}_L{lagged}_{rep}_compared_stats.npy', 'w') as f:
                        f.write("precision: ")
                        f.write(dict_results["precision"])
                        f.write("\n")
                        f.write("recall: ")
                        f.write(dict_results["recall"])
                        f.write("\n")
                        f.write("f-score: ")
                        f.write(dict_results["f-score"])
                        f.write("\n")
                        f.write("time: ")
                        f.write(return_time)
                        f.write("\n")
            counter += 1
        counter = 0

/Users/ianyuen/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:2500: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/Users/ianyuen/Documents/GitHub/RA_time_series/time_series_ra/RepLearningEngine/SINK.py:46: RuntimeWarning: invalid value encountered in true_divide
  result = np.divide(np.real(r), (np.linalg.norm(x) * np.linalg.norm(y)))


AssertionError: 